In [1]:
import ROOT
import h5py
import numpy as np

Welcome to JupyROOT 6.16/00


In [2]:
#File Energy Calo
f = h5py.File('../File_En/Outfile_CellInformation_HomDet_2to5GeV_FineGran128_V1.h5','r')
#f = h5py.File('../File_En/Outfile_CellInformation_HomDet_15to20GeV_FineGran128_V1.h5','r')
fN= h5py.File('../File_En/Outfile_MLPred_HomDet_2to5GeV_FineGran128_V1.h5','r')

In [3]:
fN.keys()

<KeysViewHDF5 ['RealRes_NeutralEnergyPred_Layer1', 'RealRes_NeutralEnergyPred_Layer2', 'RealRes_NeutralEnergyPred_Layer3', 'RealRes_NeutralEnergyPred_Layer4', 'RealRes_NeutralEnergyPred_Layer5', 'RealRes_NeutralEnergyPred_Layer6']>

In [4]:
layers=[64,32,32,16,16,8]

ev=123
lay=1
for X in range(64):
    for Y in range(64):
        NoiseT=f['RealRes_TotalEnergy_Layer'+str(lay)][ev][0]-f['RealRes_NeutralEnergy_Layer'+str(lay)][ev][0]-f['RealRes_ChargedEnergy_Layer'+str(lay)][ev][0]
        Noise=f['RealRes_Noise_Layer'+str(lay)][ev][0]
        A=NoiseT-Noise
        if abs(A[X][Y])>0.01:
            print(X, Y, A[X][Y], NoiseT[X][Y], Noise[X][Y])

In [5]:
#Total Topoclusters
gT = h5py.File('../File_Clust/Outfile_2to5GeV_TotalTopo.h5','r')
#gT = h5py.File('../File_Clust/Outfile_15to20GeV_TotalTopo.h5','r')

#gT=h5py.File('../File_Clust/A.h5','r')

#Charged Topoclusters
gCh = h5py.File('../File_Clust/Outfile_2to5GeV_ChargedTopo.h5','r')
#gCh = h5py.File('../File_Clust/Outfile_15to20GeV_ChargedTopo.h5','r')

import math

TrDi=125.

X0_ECAL = 3.897
Lambda_int_HCAL = 17.438

Total_ECAL_Length = 3 * X0_ECAL + 16 * X0_ECAL + 6 * X0_ECAL
Total_HCAL_Length = 1.5 * Lambda_int_HCAL + 4.1 * Lambda_int_HCAL + 1.8 * Lambda_int_HCAL
Total_Calo_Length = Total_ECAL_Length + Total_HCAL_Length + 1.0 # -- there is a 1 cm gap between ECAL & HCAL


zpos_ECAL1 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL/2
zpos_ECAL2 = zpos_ECAL1 + 3 * X0_ECAL/2 + 16  * X0_ECAL/2
zpos_ECAL3 = zpos_ECAL2 + 16 * X0_ECAL/2 + 6 * X0_ECAL/2

zpos_GAP = zpos_ECAL3 + 6 * X0_ECAL/2 + 1/2

zpos_HCAL1 = zpos_GAP + 1/2 + 1.5 * Lambda_int_HCAL/2
zpos_HCAL2 = zpos_HCAL1 + 1.5 * Lambda_int_HCAL/2 + 4.1 * Lambda_int_HCAL/2
zpos_HCAL3 = zpos_HCAL2 + 4.1 * Lambda_int_HCAL/2 + 1.8 * Lambda_int_HCAL/2

layer_struct = {'layer1' : 64, 'layer2' : 32, 'layer3' : 32, 'layer4' : 16, 'layer5' : 16, 'layer6' : 8}

def MakeTruthTrajectory(Theta, Phi, x_indx, y_indx) : 

    x_orig, y_orig = 0., 0.
    z_orig = -1 * (Total_Calo_Length/2 + 150.)

    if(x_indx == 5 and y_indx == 5) : x_orig, y_orig   =  10.,  10.
    if(x_indx == 5 and y_indx == -5) : x_orig, y_orig  =  10., -10.
    if(x_indx == -5 and y_indx == 5) : x_orig, y_orig  = -10.,  10.
    if(x_indx == -5 and y_indx == -5) : x_orig, y_orig = -10., -10.

    layer1_x = x_orig + (zpos_ECAL1 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer1_y = y_orig + (zpos_ECAL1 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer2_x = x_orig + (zpos_ECAL2 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer2_y = y_orig + (zpos_ECAL2 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer3_x = x_orig + (zpos_ECAL3 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer3_y = y_orig + (zpos_ECAL3 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer4_x = x_orig + (zpos_HCAL1 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer4_y = y_orig + (zpos_HCAL1 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer5_x = x_orig + (zpos_HCAL2 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer5_y = y_orig + (zpos_HCAL2 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer6_x = x_orig + (zpos_HCAL3 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer6_y = y_orig + (zpos_HCAL3 - z_orig) * math.tan(Theta) * math.sin(Phi)

    CellIndexLayer = []

    CellIndexLayer.append(  FindCellIndex( layer1_x, layer1_y, 'layer1' ) )
    CellIndexLayer.append(  FindCellIndex( layer2_x, layer2_y, 'layer2' ) )
    CellIndexLayer.append(  FindCellIndex( layer3_x, layer3_y, 'layer3' ) )
    CellIndexLayer.append(  FindCellIndex( layer4_x, layer4_y, 'layer4' ) )
    CellIndexLayer.append(  FindCellIndex( layer5_x, layer5_y, 'layer5' ) )
    CellIndexLayer.append(  FindCellIndex( layer6_x, layer6_y, 'layer6' ) )

    return np.array(CellIndexLayer)


def FindCellIndex(tr_x, tr_y, layer_name) : 
        
    x_idx = int( (tr_x + TrDi/2.)/TrDi * layer_struct[layer_name] )
    y_idx = int( (tr_y + TrDi/2.)/TrDi * layer_struct[layer_name] )
    
    return np.array([x_idx, y_idx])





import math

layer_struct = {'layer1' : 64, 'layer2' : 32, 'layer3' : 32, 'layer4' : 16, 'layer5' : 16, 'layer6' : 8}


def FindCellIndexA(tr_x, tr_y, layer_name) : 
    
    cell_x_1 = np.modf(tr_x/(1250/layer_struct[layer_name])+ layer_struct[layer_name]/2 )[1].astype(int)
    cell_y_1 = np.modf(tr_y/(1250/layer_struct[layer_name])+ layer_struct[layer_name]/2 )[1].astype(int)
    
    return np.array([cell_x_1, cell_y_1])

def TrackAlt(trk_X_pos,trk_Y_pos,trk_Theta,trk_Phi):

    X0_ECAL = 3.897
    Lambda_int_HCAL = 17.438

    Total_ECAL_Length = 3 * X0_ECAL + 16 * X0_ECAL + 6 * X0_ECAL
    Total_HCAL_Length = 1.5 * Lambda_int_HCAL + 4.1 * Lambda_int_HCAL + 1.8 * Lambda_int_HCAL
    Total_Calo_Length = Total_ECAL_Length + Total_HCAL_Length + 1.0 # -- there is a 1 cm gap between ECAL & HCAL


    zpos_ECAL1 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL/2
    zpos_ECAL2 = zpos_ECAL1 + 3 * X0_ECAL/2 + 16 *  X0_ECAL/2
    zpos_ECAL3 = zpos_ECAL2 + 16 * X0_ECAL/2 + 6 * X0_ECAL/2

    zpos_GAP = zpos_ECAL3 + 6 * X0_ECAL/2 + 1/2

    zpos_HCAL1 = zpos_GAP + 1/2 + 1.5 * Lambda_int_HCAL/2
    zpos_HCAL2 = zpos_HCAL1 + 1.5 * Lambda_int_HCAL/2 + 4.1 * Lambda_int_HCAL/2
    zpos_HCAL3 = zpos_HCAL2 + 4.1 * Lambda_int_HCAL/2 + 1.8 * Lambda_int_HCAL/2
    z_orig = -1 * (Total_Calo_Length/2 + 150)
    trk_X_n = trk_X_pos/10.0 - np.abs(-zpos_ECAL1+z_orig+3/2*X0_ECAL)*np.tan(trk_Theta)*np.cos(trk_Phi)
    trk_Y_n = trk_Y_pos/10.0 - np.abs(-zpos_ECAL1+z_orig+3/2*X0_ECAL)*np.tan(trk_Theta)*np.sin(trk_Phi)
    x_1 = (trk_X_n + np.abs(-zpos_ECAL1+z_orig+3/2*X0_ECAL)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_1 = (trk_Y_n + np.abs(-zpos_ECAL1+z_orig+3/2*X0_ECAL)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_2 = (trk_X_n + np.abs(-zpos_ECAL2+z_orig+16/2*X0_ECAL)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_2 = (trk_Y_n + np.abs(-zpos_ECAL2+z_orig+16/2*X0_ECAL)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_3 = (trk_X_n + np.abs(-zpos_ECAL3+z_orig+6/2*X0_ECAL)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_3 = (trk_Y_n + np.abs(-zpos_ECAL3+z_orig+6/2*X0_ECAL)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_4 = (trk_X_n + np.abs(-zpos_HCAL1+z_orig+1.5*Lambda_int_HCAL/2)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_4 = (trk_Y_n + np.abs(-zpos_HCAL1+z_orig+1.5*Lambda_int_HCAL/2)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_5 = (trk_X_n + np.abs(-zpos_HCAL2+z_orig+4.1*Lambda_int_HCAL/2)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_5 = (trk_Y_n + np.abs(-zpos_HCAL2+z_orig+4.1*Lambda_int_HCAL/2)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_6 = (trk_X_n + np.abs(-zpos_HCAL3+z_orig+1.8*Lambda_int_HCAL/2)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_6 = (trk_Y_n + np.abs(-zpos_HCAL3+z_orig+1.8*Lambda_int_HCAL/2)*np.tan(trk_Theta)*np.sin(trk_Phi))*10

    
    l1_idx = FindCellIndexA(x_1, y_1, 'layer1')
    l2_idx = FindCellIndexA(x_2, y_2, 'layer2')
    l3_idx = FindCellIndexA(x_3, y_3, 'layer3')
    l4_idx = FindCellIndexA(x_4, y_4, 'layer4')
    l5_idx = FindCellIndexA(x_5, y_5, 'layer5')
    l6_idx = FindCellIndexA(x_6, y_6, 'layer6')
    
    return np.array([l1_idx, l2_idx, l3_idx, l4_idx, l5_idx, l6_idx])

In [6]:
layer_struct = {'layer1' : 64, 'layer2' : 32, 'layer3' : 32, 'layer4' : 16, 'layer5' : 16, 'layer6' : 8}

def FindCellIndex(tr_x, tr_y, layer_name) : 
    
    add_x, add_y = 0,0
    if( int(tr_x/(1250/128)+ 64) % int( 128/ layer_struct[layer_name]) != 0) : add_x += 1
    if( int(tr_y/(1250/128)+ 64) % int( 128/ layer_struct[layer_name]) != 0) : add_y += 1
    
    cell_x_1 = np.modf( (tr_x/(1250/128)+ 64) * layer_struct[layer_name]/128 )[1].astype(int) + add_x  
    cell_y_1 = np.modf( (tr_y/(1250/128)+ 64) * layer_struct[layer_name]/128 )[1].astype(int) + add_y
    return np.array([cell_x_1, cell_y_1])

def MakeTruthTrajectory(trk_Theta,trk_Phi,trk_X_pos,trk_Y_pos):

    X0_ECAL = 3.897
    Lambda_int_HCAL = 17.438

    Total_ECAL_Length = 3 * X0_ECAL + 16 * X0_ECAL + 6 * X0_ECAL
    Total_HCAL_Length = 1.5 * Lambda_int_HCAL + 4.1 * Lambda_int_HCAL + 1.8 * Lambda_int_HCAL
    Total_Calo_Length = Total_ECAL_Length + Total_HCAL_Length + 1.0 # -- there is a 1 cm gap between ECAL & HCAL


    zpos_ECAL1 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL/2
    zpos_ECAL2 = zpos_ECAL1 + 3 * X0_ECAL/2 + 16  * X0_ECAL/2
    zpos_ECAL3 = zpos_ECAL2 + 16 * X0_ECAL/2 + 6 * X0_ECAL/2

    zpos_GAP = zpos_ECAL3 + 6 * X0_ECAL/2 + 1/2

    zpos_HCAL1 = zpos_GAP + 1/2 + 1.5 * Lambda_int_HCAL/2
    zpos_HCAL2 = zpos_HCAL1 + 1.5 * Lambda_int_HCAL/2 + 4.1 * Lambda_int_HCAL/2
    zpos_HCAL3 = zpos_HCAL2 + 4.1 * Lambda_int_HCAL/2 + 1.8 * Lambda_int_HCAL/2
    z_orig = -1 * (Total_Calo_Length/2 + 150)
    trk_X_n = trk_X_pos/10.0 - np.abs(-zpos_ECAL1+z_orig+3/2*X0_ECAL)*np.tan(trk_Theta)*np.cos(trk_Phi)
    trk_Y_n = trk_Y_pos/10.0 - np.abs(-zpos_ECAL1+z_orig+3/2*X0_ECAL)*np.tan(trk_Theta)*np.sin(trk_Phi)
    
    
    x_1 = (trk_X_n + np.abs(-zpos_ECAL1+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_1 = (trk_Y_n + np.abs(-zpos_ECAL1+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_2 = (trk_X_n + np.abs(-zpos_ECAL2+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_2 = (trk_Y_n + np.abs(-zpos_ECAL2+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_3 = (trk_X_n + np.abs(-zpos_ECAL3+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_3 = (trk_Y_n + np.abs(-zpos_ECAL3+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_4 = (trk_X_n + np.abs(-zpos_HCAL1+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_4 = (trk_Y_n + np.abs(-zpos_HCAL1+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_5 = (trk_X_n + np.abs(-zpos_HCAL2+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_5 = (trk_Y_n + np.abs(-zpos_HCAL2+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    x_6 = (trk_X_n + np.abs(-zpos_HCAL3+z_orig)*np.tan(trk_Theta)*np.cos(trk_Phi))*10
    y_6 = (trk_Y_n + np.abs(-zpos_HCAL3+z_orig)*np.tan(trk_Theta)*np.sin(trk_Phi))*10
    
    l1_idx = FindCellIndex(x_1, y_1, 'layer1')
    l2_idx = FindCellIndex(x_2, y_2, 'layer2')
    l3_idx = FindCellIndex(x_3, y_3, 'layer3')
    l4_idx = FindCellIndex(x_4, y_4, 'layer4')
    l5_idx = FindCellIndex(x_5, y_5, 'layer5')
    l6_idx = FindCellIndex(x_6, y_6, 'layer6')
    
    return np.array([l1_idx, l2_idx, l3_idx, l4_idx, l5_idx, l6_idx]).astype(int)

In [7]:
def Count_Topo(gio, events,key):
    Num_Topo=np.zeros(events)
    Ne90 = np.zeros(10)
    NN = []
    print("starting loop")
    for ev in range(events):
        if (ev+1)%100==0: print('still looping',ev)        
            
        
        #sorted energies in topoclusters
        N_Cl = -1
        for lay in range(1,7):
            Topo = gio[str(key)+str(lay)][ev]
            M=np.amax(Topo)
            #print(M)
            if M>N_Cl:
                N_Cl = M
            
        NN.append(int(N_Cl))
        
    return NN


In [8]:
NTT=len(gT['TopoClusters1'])
NCT=len(gCh['TopoClusters1'])

N_ClusT=Count_Topo(gT, NTT,"TopoClusters" )
N_ClusC=Count_Topo(gCh,NCT,"TopoClusters" )

starting loop
still looping 99
still looping 199
still looping 299
still looping 399
still looping 499
still looping 599
still looping 699
still looping 799
still looping 899
still looping 999
still looping 1099
still looping 1199
still looping 1299
still looping 1399
still looping 1499
still looping 1599
still looping 1699
still looping 1799
still looping 1899
still looping 1999
still looping 2099
still looping 2199
still looping 2299
still looping 2399
still looping 2499
still looping 2599
still looping 2699
still looping 2799
still looping 2899
still looping 2999
still looping 3099
still looping 3199
still looping 3299
still looping 3399
still looping 3499
still looping 3599
still looping 3699
still looping 3799
still looping 3899
still looping 3999
still looping 4099
still looping 4199
still looping 4299
still looping 4399
still looping 4499
still looping 4599
still looping 4699
still looping 4799
still looping 4899
still looping 4999
still looping 5099
still looping 5199
still loo

In [9]:
#N_ClusT, N_ClusC

for i, x in enumerate(N_ClusT):
    if x>7: print(i)

In [10]:
def histos(label):

    h = []
    
    h_E_L0= ROOT.TH2D("L0"+label,"L0"+label,64,1,65,64,1,65)
    h_E_L1= ROOT.TH2D("L1"+label,"L1"+label,32,1,33,32,1,33)
    h_E_L2= ROOT.TH2D("L2"+label,"L2"+label,32,1,33,32,1,33)
    h_E_L3= ROOT.TH2D("L3"+label,"L3"+label,16,1,17,16,1,17)
    h_E_L4= ROOT.TH2D("L4"+label,"L4"+label,16,1,17,16,1,17)
    h_E_L5= ROOT.TH2D("L5"+label,"L5"+label,8,1,9,8,1,9)
    
    h_E_L0.Reset()


    h.append(h_E_L0)
    h.append(h_E_L1)
    h.append(h_E_L2)
    h.append(h_E_L3)
    h.append(h_E_L4)
    h.append(h_E_L5)

    return h


In [29]:
import math

def EventDisplay(part, Clus, ev, n_clus):
    
    h_E_tot = histos( str(part) + "_" + str(Clus) + "En" )

    i = 0
    h_clus= [ histos( str(part) + "_" + str(Clus) + "Topo_Clus"+str(i) ) for i in range(n_clus)]

    #print(len(h_clus))
    h_Traj= histos( "Mark_Traj" + str(ev) )
    
    layers=[64,32,32,16,16,8]

    C1=f["Trk_Theta"]
    C2=f["Trk_Phi"]
    C3=f["Trk_X_pos"]
    C4=f["Trk_Y_pos"]
    
    Traj_Mark = MakeTruthTrajectory(C1[ev], C2[ev],C3[ev], C4[ev])

    print(C1[ev], C2[ev], C3[ev], C4[ev])
    
    
    for lay in range(1,7):
        
        if Clus=="Total":
            Tot_En_Distr=f['RealRes_'+str(part)+'Energy_Layer'+str(lay)][ev][0]
            #Tot_En_Distr=fN['RealRes_NeutralEnergyPred_Layer'+str(lay)][ev][0]
            Clust_Cell=gT["TopoClusters"+str(lay)][ev][0]
            
        if Clus=="Charged":
            Tot_En_Distr=f['RealRes_ChargedEnergy_Layer'+str(lay)][ev][0]#+f['RealRes_Noise_Layer'+str(lay)][ev][0]
            Clust_Cell=gCh["TopoClusters"+str(lay)][ev][0]
            
        h_Traj[lay-1].SetBinContent(int(Traj_Mark[lay-1][0])+1 , int(Traj_Mark[lay-1][1])+1 ,10)
        
        for X in range(layers[lay-1]):
            for Y in range(layers[lay-1]):
                if Tot_En_Distr[X][Y]>1.:
                    h_E_tot[lay-1].SetBinContent(X+1,Y+1, Tot_En_Distr[X][Y])
                if Tot_En_Distr[X][Y]<=1.:
                    h_E_tot[lay-1].SetBinContent(X+1,Y+1, 1)
                TopoI=int(Clust_Cell[X][Y])
                if TopoI!=0:
                    h_clus[TopoI-1][lay-1].SetBinContent(X+1,Y+1,100)

    layers=[64,32,32,16, 16, 8]
    col=[ 2, 1, 3, 6,  7 ,8, 9, 11]
    #col=[i for i in range(n_clus)]
    can=ROOT.TCanvas( "ev_"+str(ev), "ev_"+str(ev),1600,900);
    #can.SetLogz()
    can.Divide(3,2);
    for j in range(6):
        can.cd(j+1)
        ROOT.gPad.SetLogz()
        ROOT.gStyle.SetOptStat(0)
        h_E_tot[j].Draw("colz")

        for cl in range(n_clus):
            #print(cl,j,n_clus[ev])
            h_clus[cl][j].SetLineColor(col[cl])
            h_clus[cl][j].SetFillColor(col[cl])
            h_clus[cl][j].SetLineWidth(3)            
            h_clus[cl][j].SetContour(1);
            h_clus[cl][j].SetMarkerStyle(2)
            h_clus[cl][j].SetMarkerColor(col[cl])
            h_clus[cl][j].SetMarkerSize(0.5)
            
            h_clus[cl][j].Draw("cont3 same");

        h_Traj[j].SetMarkerStyle(3)
        h_Traj[j].SetMarkerColor(1)
        h_Traj[j].SetMarkerSize(10)
        h_Traj[j].SetFillColor(1)
        h_Traj[j].SetLineColor(1)
        h_Traj[j].Draw("box same")
    
        #h_E_tot[j].GetZaxis().SetRangeUser(0.5, 4000)
    can.Print("../DATA_PROCESSING/2to5Pred/ED/AMATruth_ED_"+str(ev)+str(part)+"Energy_"+str(Clus)+"Topo.png")
    #+"Predicted"

In [31]:
EV=5432
print("Number of topoclusters", N_ClusC[EV])
EventDisplay("Charged", "Charged",EV, N_ClusC[EV])

Number of topoclusters 1
0.0434922 3.5639381 40.456635 73.24189


Info in <TCanvas::Print>: png file ../DATA_PROCESSING/2to5Pred/ED/AMATruth_ED_5432ChargedEnergy_ChargedTopo.png has been created
